In [1]:
import numpy as np
import cv2 as cv
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F


In [2]:
path = "resources/myData"
testRatio = 0.2
valRatio = 0.2

In [3]:
myList = os.listdir(path)
nClasses = len(myList) 
images = []
classes = []

In [9]:
for x in range(0, nClasses):
    myPicList = os.listdir(path+"/"+str(x))
    for y in myPicList:
        curImg = cv.imread(path+"/"+str(x)+"/"+y)
        curImg = cv.resize(curImg, (28, 28))
        images.append(curImg)
        classes.append(x)
    print(x)

0
1
2
3
4
5
6
7
8
9


In [72]:
images = np.array(images)
classes = np.array(classes)

In [17]:
images[0]

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [74]:
### Split the data into training and testing

X_train, X_test, y_train, y_test = train_test_split(images, classes, test_size=testRatio, random_state=42)

In [75]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=valRatio, random_state=42)

In [76]:
X_train.shape

(6502, 28, 28, 3)

In [77]:
nSamples = []

for x in range(0, nClasses):
    nSamples.append(len(np.where(y_train == x)[0]))

print(nSamples)

[641, 659, 650, 647, 636, 646, 678, 631, 636, 678]


In [78]:
def preProcessing(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.equalizeHist(img)
    img = img/255
    return img

In [79]:
X_train = np.array(list(map(preProcessing, X_train)))

In [80]:
X_test = np.array(list(map(preProcessing, X_test)))
X_val = np.array(list(map(preProcessing, X_val)))

In [81]:
# X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
# X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
# X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)

In [100]:
train_data = []
for i in range(0, len(X_train)):
    train_data.append([X_train[i], y_train[i]])

trainloader = DataLoader(train_data, batch_size=32, shuffle=True)

In [101]:
test_data = []
for i in range(0, len(X_test)):
    test_data.append([X_test[i], y_test[i]])

testloader = DataLoader(test_data, batch_size=6, shuffle=True)

In [117]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
type(images[0])

torch.Tensor

In [90]:
# Hyperparameters for our network
input_size   = 784
hidden_sizes = [128, 64]
output_size  = 10

# Build a feed-forward network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU())  # on device cpu
        self.fc2 = nn.Sequential(
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU())  # on device "cuda:1"
        self.fc3 = nn.Sequential(
            nn.Linear(hidden_sizes[1], output_size))  # on device "cuda:0"
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

In [92]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [98]:
epochs = 50
print_every = 40

for e in range(epochs):
    running_loss = 0
    print(f"Epoch: {e+1}/{epochs}")

    for i, (images, labels) in enumerate(iter(trainloader)):

        # Flatten MNIST images into a 784 long vector
        images.resize_(images.size()[0], 784)
        
        optimizer.zero_grad()
        
        output = model.forward(images.float())   # 1) Forward pass
        loss = criterion(output, labels) # 2) Compute loss
        loss.backward()                  # 3) Backward pass
        optimizer.step()                 # 4) Update model
        
        running_loss += loss.item()
        
        if i % print_every == 0:
            print(f"\tIteration: {i}\t Loss: {running_loss/print_every:.4f}")
            running_loss = 0

torch.save(model, "output/dig_model.pt")

Epoch: 1/50
	Iteration: 0	 Loss: 0.0187
	Iteration: 40	 Loss: 0.6610
	Iteration: 80	 Loss: 0.6273
	Iteration: 120	 Loss: 0.6369
	Iteration: 160	 Loss: 0.6111
	Iteration: 200	 Loss: 0.5891
Epoch: 2/50
	Iteration: 0	 Loss: 0.0118
	Iteration: 40	 Loss: 0.5798
	Iteration: 80	 Loss: 0.5680
	Iteration: 120	 Loss: 0.5194
	Iteration: 160	 Loss: 0.5264
	Iteration: 200	 Loss: 0.4832
Epoch: 3/50
	Iteration: 0	 Loss: 0.0118
	Iteration: 40	 Loss: 0.5022
	Iteration: 80	 Loss: 0.4867
	Iteration: 120	 Loss: 0.4810
	Iteration: 160	 Loss: 0.4497
	Iteration: 200	 Loss: 0.4310
Epoch: 4/50
	Iteration: 0	 Loss: 0.0116
	Iteration: 40	 Loss: 0.4162
	Iteration: 80	 Loss: 0.4577
	Iteration: 120	 Loss: 0.4113
	Iteration: 160	 Loss: 0.4176
	Iteration: 200	 Loss: 0.4018
Epoch: 5/50
	Iteration: 0	 Loss: 0.0118
	Iteration: 40	 Loss: 0.3887
	Iteration: 80	 Loss: 0.4044
	Iteration: 120	 Loss: 0.3831
	Iteration: 160	 Loss: 0.3884
	Iteration: 200	 Loss: 0.3609
Epoch: 6/50
	Iteration: 0	 Loss: 0.0076
	Iteration: 40	 Loss

In [102]:
model = torch.load('output/dig_model.pt')
model.eval()

total = 0
correct = 0
for i, (images, labels) in enumerate(iter(testloader)):
    images.resize_(images.size()[0], 784)
    print(labels)
	#print(labels)
    preds = model.forward(images.float())
    _, prediction = torch.max(preds, dim = 1)
    count = sum(prediction == labels).item()
    print(count)
		#if random.random() < 0.80:
		#	print("PREDICTIONS : ", prediction)
		#	print("LABELS : ", labels)
		#break
    total += labels.size(0)
    correct += count
accuracy = correct/total
print(accuracy)

tensor([5, 3, 1, 9, 3, 4])
6
tensor([9, 6, 9, 3, 2, 1])
6
tensor([7, 4, 8, 0, 2, 4])
6
tensor([6, 7, 8, 1, 6, 3])
6
tensor([5, 1, 7, 5, 4, 7])
6
tensor([2, 9, 0, 3, 5, 8])
6
tensor([7, 3, 0, 2, 2, 5])
6
tensor([8, 6, 8, 0, 4, 4])
6
tensor([5, 1, 1, 0, 2, 9])
6
tensor([8, 5, 1, 8, 1, 4])
6
tensor([8, 6, 4, 2, 6, 7])
6
tensor([8, 3, 7, 3, 0, 2])
5
tensor([1, 4, 9, 2, 1, 3])
5
tensor([0, 9, 7, 3, 0, 9])
5
tensor([6, 3, 5, 1, 3, 3])
6
tensor([8, 9, 2, 8, 1, 4])
5
tensor([6, 3, 8, 4, 8, 9])
6
tensor([9, 2, 2, 6, 3, 2])
6
tensor([3, 6, 2, 9, 2, 9])
6
tensor([0, 0, 9, 7, 7, 2])
6
tensor([8, 1, 1, 0, 4, 7])
5
tensor([4, 0, 4, 7, 1, 2])
6
tensor([6, 5, 6, 7, 1, 8])
6
tensor([9, 3, 0, 1, 4, 5])
6
tensor([7, 9, 9, 6, 8, 3])
6
tensor([7, 5, 9, 2, 4, 6])
5
tensor([6, 9, 4, 6, 6, 9])
6
tensor([0, 9, 2, 7, 4, 7])
6
tensor([2, 8, 2, 6, 0, 8])
6
tensor([5, 3, 2, 2, 6, 0])
6
tensor([5, 7, 1, 8, 1, 2])
6
tensor([2, 7, 6, 2, 0, 9])
6
tensor([5, 3, 5, 1, 9, 1])
6
tensor([1, 9, 4, 7, 6, 5])
5
tensor([5, 8, 